In [1]:


import sys, os, re, csv, codecs, numpy as np, pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers



/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2018-03-20 16:05:04--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2018-03-20 16:05:04--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  19.0MB/s    in 41s     

2018-03-20 16:05:45 (20.1 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [3]:
import zipfile
zip_ref = zipfile.ZipFile('glove.6B.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

In [4]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8" --header="Accept-Language: en-GB,en-US;q=0.8,en;q=0.6" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1521812573&Signature=SHWEQ%2BJjP5RTH7WdHtCnUFq6Kjv5e%2FCaxfU8yvk8UmIW04XGVqW9979JQ0kAh3BmamwxT8iysjVuIz2DsrTDQLuFrUWYwh34wRi5cjYzrdkgYmWo5SIDHHkd7%2FHPcOXCF0v5OywlP4wB0RCWPvxaaQyXU%2BwJuhRS5LLrrKBYI%2Bua8V3BBUXpJf%2B0jh7UwpnSQmc5brW8l56cIg2LhaqzWC4C3bi2CvOzUmBELI8Bhem3anQYQdN3kKhWxm2ij3Q2GHhMBia%2BOd0Oqk1%2BuLvN07Zlbe%2FF3yDRWTmF%2BFLoCzrOWoGWlB8qfj90dxXBYN8NVLBfrF7wtmCUUVhqVumJ5w%3D%3D" -O "train.csv.zip" -c

--2018-03-20 16:06:02--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/train.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1521812573&Signature=SHWEQ%2BJjP5RTH7WdHtCnUFq6Kjv5e%2FCaxfU8yvk8UmIW04XGVqW9979JQ0kAh3BmamwxT8iysjVuIz2DsrTDQLuFrUWYwh34wRi5cjYzrdkgYmWo5SIDHHkd7%2FHPcOXCF0v5OywlP4wB0RCWPvxaaQyXU%2BwJuhRS5LLrrKBYI%2Bua8V3BBUXpJf%2B0jh7UwpnSQmc5brW8l56cIg2LhaqzWC4C3bi2CvOzUmBELI8Bhem3anQYQdN3kKhWxm2ij3Q2GHhMBia%2BOd0Oqk1%2BuLvN07Zlbe%2FF3yDRWTmF%2BFLoCzrOWoGWlB8qfj90dxXBYN8NVLBfrF7wtmCUUVhqVumJ5w%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.15.80, 2607:f8b0:4004:811::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.15.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27619914 (26M) [application/zip]
Saving to: ‘train.csv.zip’

train.csv.zip       100%[===================>]  26.34M   103MB/s    in 0.3s    

2018-03-20 16:06:03 (103 MB/s) - ‘train.

In [5]:
import zipfile
zip_ref = zipfile.ZipFile('train.csv.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

In [6]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8" --header="Accept-Language: en-GB,en-US;q=0.8,en;q=0.6" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1521812594&Signature=iT1U%2FpLBN9XqCKtb2uwGE2jcLuN%2FWSfMt%2F%2FcQUwcfbe4EJ%2Biu07enqKtYZGtVfb4YQEicofRmEkZBnrGCPt%2BiZxi9QWnIe5vnhkCII%2BoCwl%2FsBCqKXrNPmno2jfi2oWk70ME2Ajci04j8xCNynF8I%2FqjQp2VAwxE0wJ1ii0jo5V8PQ7vrJoLH6rmsKgdDbXA2TpFejMdT2g5ZbMTy6LTtx9nZ0YuTnvUvrofLM5BtmZPGiyW%2BXUiZOKT0Vkz69SsSwMaQl2lRQTgxc2pOHus%2FVJYGrjoMf%2ByRV%2F1tpKhMKjzRrWqXSkYE7%2BznqI7VjEGzfisLlwzhYSX%2FIyXTrSjUw%3D%3D" -O "test.csv.zip" -c

--2018-03-20 16:06:04--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/test.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1521812594&Signature=iT1U%2FpLBN9XqCKtb2uwGE2jcLuN%2FWSfMt%2F%2FcQUwcfbe4EJ%2Biu07enqKtYZGtVfb4YQEicofRmEkZBnrGCPt%2BiZxi9QWnIe5vnhkCII%2BoCwl%2FsBCqKXrNPmno2jfi2oWk70ME2Ajci04j8xCNynF8I%2FqjQp2VAwxE0wJ1ii0jo5V8PQ7vrJoLH6rmsKgdDbXA2TpFejMdT2g5ZbMTy6LTtx9nZ0YuTnvUvrofLM5BtmZPGiyW%2BXUiZOKT0Vkz69SsSwMaQl2lRQTgxc2pOHus%2FVJYGrjoMf%2ByRV%2F1tpKhMKjzRrWqXSkYE7%2BznqI7VjEGzfisLlwzhYSX%2FIyXTrSjUw%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.15.80, 2607:f8b0:4004:811::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.15.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24577258 (23M) [application/zip]
Saving to: ‘test.csv.zip’

test.csv.zip        100%[===================>]  23.44M  99.2MB/s    in 0.2s    

2018-03-20 16:06:04 (99.2 MB/s) 

In [7]:
import zipfile
zip_ref = zipfile.ZipFile('test.csv.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

In [8]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.81 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8" --header="Accept-Language: en-GB,en-US;q=0.8,en;q=0.6" "https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/sample_submission.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1521813930&Signature=Sy%2B7sOWrrJueDZCRmfQj2mF54voR8878mK07%2FoCf3%2Bqa08zU6S0azAYO7C3k3YI6Yt39dKRgM12Z96wZLcslW96lUxy22r0BbnYzjwr8gR9WgnpevM3NyUOp6HTsvYFNAsHeCXY5hzOjtfTaxie9DWCDUIvVroiIamoS77DOyHcGPbwfgaiqRjlMdmOZOSiE7hjp2ntYaKspnB5VyUR30iZiQ%2BNjsfqYDNUoeVYeYKlIfTRIb4FbGc1Q8Kzetj58ptDtjadOWxlyV5F8H6zjNfGCVkAtuj9uO1KFgQatp18tjN3grXWoaJn6pjghhmVzHRgUmKewPPAwrfAGDPEK%2Fg%3D%3D" -O "sample_submission.csv.zip" -c

--2018-03-20 16:06:05--  https://storage.googleapis.com/kaggle-competitions-data/kaggle/8076/sample_submission.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1521813930&Signature=Sy%2B7sOWrrJueDZCRmfQj2mF54voR8878mK07%2FoCf3%2Bqa08zU6S0azAYO7C3k3YI6Yt39dKRgM12Z96wZLcslW96lUxy22r0BbnYzjwr8gR9WgnpevM3NyUOp6HTsvYFNAsHeCXY5hzOjtfTaxie9DWCDUIvVroiIamoS77DOyHcGPbwfgaiqRjlMdmOZOSiE7hjp2ntYaKspnB5VyUR30iZiQ%2BNjsfqYDNUoeVYeYKlIfTRIb4FbGc1Q8Kzetj58ptDtjadOWxlyV5F8H6zjNfGCVkAtuj9uO1KFgQatp18tjN3grXWoaJn6pjghhmVzHRgUmKewPPAwrfAGDPEK%2Fg%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.15.80, 2607:f8b0:4004:811::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.15.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1464269 (1.4M) [application/zip]
Saving to: ‘sample_submission.csv.zip’

sample_submission.c 100%[===================>]   1.40M  --.-KB/s    in 0.07s   

2018-03-20 16:06:05 (19.6 

In [9]:
import zipfile
zip_ref = zipfile.ZipFile('sample_submission.csv.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

In [10]:
embed_size = 50 # how big is each word vector
max_features = 150000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 150 # max number of words in a comment to use


In [11]:


#path = '../input/'
#comp = 'jigsaw-toxic-comment-classification-challenge/'
EMBEDDING_FILE='./glove.6B.50d.txt'
TRAIN_DATA_FILE='./train.csv'
TEST_DATA_FILE='./test.csv'



In [12]:


train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

list_sentences_train = train["comment_text"].fillna("_na_").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("_na_").values



In [13]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

In [14]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(EMBEDDING_FILE))

In [15]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std


(0.020940464, 0.6441042)

In [16]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [26]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(6, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model.fit(X_t, y, batch_size=512, epochs=1);

Epoch 1/1
159571/159571 [==============================] - 161s 1ms/step - loss: 0.1148 - acc: 0.9672


In [ ]:
model.fit(X_t, y, batch_size=512, epochs=1);

Epoch 1/1
142336/159571 [=========================>....] - ETA: 17s - loss: 0.0554 - acc: 0.9802

In [ ]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission[list_classes] = y_test
sample_submission.to_csv('submission-lstm-2.csv', index=False)


In [ ]:
from IPython.lib.display import FileLink

In [8]:
FileLink("train.csv")

/home/ubuntu/project/train.csv

In [ ]:
FileLink("submission-lstm-2.csv")

In [ ]:
FileLink("model.json")

In [ ]:
FileLink("model.h5")